In [12]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import folium 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [22]:
Link = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(Link, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    if row != [] and row[1] != "Not assigned":
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

df_Canada= pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df_Canada["PostalCode"] = df_Canada["PostalCode"].str.replace("\n","")
df_Canada ["Borough"] = df_Canada["Borough"].str.replace("\n","")
df_Canada["Neighborhood"] = df_Canada["Neighborhood"].str.replace("\n","")
df_Canada.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [50]:
df_LatLong = pd.read_csv("K:\Geospatial_Coordinates.csv")
df_Toronto = pd.merge(df_Canada, df_LatLong, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
df_Torontonew = df_Toronto.dropna( axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Torontonew.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
2,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
3,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


In [51]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [26]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_Toronto

In [52]:
for lat, lng, borough, neighborhood in zip(
        df_Torontonew['Latitude'], 
        df_Torontonew['Longitude'], 
        df_Torontonew['Borough'], 
        df_Torontonew['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
map_Toronto